# **Personal Details**
**Full name**: Konstantinos Gerogiannis  
**Studies**: Electrical and Computer Engineering, AUTh  
**Email**: kostas.gerogiannis04@gmail.com


## **Importing libraries**

We follow a similar code flow as in the previous Colab, where we used MLP neural networks. 
Initially, we import useful libraries:
* the matplotlib library for plotting graphs,
* the time library for calculating code execution time,
* the numpy library for fast mathematical operations and handling large volumes of data,
* the random library for random operations in the program,
* the idx2numpy library for the appropriate conversion of input data to numpy format.

In [1]:
!pip install idx2numpy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import idx2numpy
import time

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for idx2numpy: filename=idx2numpy-1.2.3-py3-none-any.whl size=7919 sha256=196d1d86c0816aa7e68e3b268d7818eab0ae986ff99b84461d14740e25bef66b
  Stored in directory: /root/.cache/pip/wheels/27/40/a8/6447ee4a00cb87e2084e1ef1df5c38433720cc1090be082842
Successfully built idx2numpy


## **Get MNIST dataset from web**


In [2]:
import urllib.request
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz', 'train-images-idx3-ubyte.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz', 'train-labels-idx1-ubyte.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz', 't10k-images-idx3-ubyte.gz')
urllib.request.urlretrieve('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz', 't10k-labels-idx1-ubyte.gz')

import gzip
import shutil

with gzip.open('train-images-idx3-ubyte.gz', 'rb') as f_in:
    with open('train-images-idx3-ubyte', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
with gzip.open('train-labels-idx1-ubyte.gz', 'rb') as f_in:
    with open('train-labels-idx1-ubyte', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
with gzip.open('t10k-images-idx3-ubyte.gz', 'rb') as f_in:
    with open('t10k-images-idx3-ubyte', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)
with gzip.open('t10k-labels-idx1-ubyte.gz', 'rb') as f_in:
    with open('t10k-labels-idx1-ubyte', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

!rm 'train-images-idx3-ubyte.gz'
!rm 'train-labels-idx1-ubyte.gz'
!rm 't10k-images-idx3-ubyte.gz'
!rm 't10k-labels-idx1-ubyte.gz'

## **Preparing the data**
We store the data in separate training and validation sets and print their dimensions. In this Colab, we also perform additional scaling of the values to the range [0,1] from the original range [0,255]. Furthermore, we create only 2 classes instead of 10:
- If the number is odd, it belongs to class 1.
- If it is even, it belongs to class 0.

In [3]:
train_images = idx2numpy.convert_from_file('train-images-idx3-ubyte')
train_labels = idx2numpy.convert_from_file('train-labels-idx1-ubyte')
test_images = idx2numpy.convert_from_file('t10k-images-idx3-ubyte')
test_labels = idx2numpy.convert_from_file('t10k-labels-idx1-ubyte')

dims = np.shape(train_images)
X_train = train_images.reshape(dims[0],dims[1]*dims[2])
y_train = train_labels
print('train set dims: ',dims)
dims = np.shape(test_images)
X_test = test_images.reshape(dims[0],dims[1]*dims[2])
y_test = test_labels
print('test set dims: ',dims)

# scaling to [0,1]
X_train = X_train/255.0
X_test = X_test/255.0

train set dims:  (60000, 28, 28)
test set dims:  (10000, 28, 28)


In [14]:
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

def radial_basis_function(x,c,sigma,f_name):
  if f_name == 'gaussian':
    return np.exp(-1/(2*sigma**2) * np.linalg.norm(x-c)**2)


class RBFNetwork(object):
  def __init__(self, k=10, learning_rate=0.01, epochs=10, rbf=radial_basis_function):
    self.k = k
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.rbf = radial_basis_function
    self.w = np.random.randn(k)
    self.b = np.random.randn(1)

  def fit(self,X,y):
    kmeans = KMeans(n_clusters=10).fit(X_train)
    self.centers = kmeans.cluster_centers_
    print('centers',np.shape(self.centers))
    max_distance = np.max([np.abs(c1-c2) for c1 in self.centers for c2 in self.centers])
    self.s = np.repeat(max_distance/np.sqrt(2*self.k), self.k)

    # training
    for epoch in range(self.epochs):
        for i in range(X.shape[0]):
            # forward pass
            a = np.array([self.rbf(X[i], c, s,'gaussian') for c, s, in zip(self.centers, self.s)])
            F = a.T.dot(self.w) + self.b
            
            loss = (y[i] - F).flatten() ** 2
            # backward pass
            error = -(y[i] - F).flatten()
            print("error",error)
            # online update
            self.w = self.w - self.learning_rate * a*error
            self.b = self.b - self.learning_rate * error
        print('Loss: {0:.2f}'.format(loss[0]))

  def predict(self, X):
      y_pred = []
      for i in range(X.shape[0]):
          a = np.array([self.rbf(X[i], c, s,'gaussian') for c, s, in zip(self.centers, self.s)])
          F = np.dot(np.transpose(a),self.w) + self.b
          y_pred.append(F)
      return np.array(y_pred)

rbfnetwork = RBFNetwork()
rbfnetwork.fit(X_train,y_train)
y_pred = rbfnetwork.predict(X_test)

print('accuracy score: ',accuracy_score(y_test,y_pred))




Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
error [2.22367694]
error [-3.79855983]
error [-2.76057423]
error [0.26703151]
error [0.2643612]
error [4.26171759]
error [-4.78089959]
error [1.26690941]
error [-4.74575969]
error [-0.69830209]
error [2.30868093]
error [-1.71440588]
error [-4.69726182]
error [3.3497108]
error [-3.68378631]
error [-1.64694845]
error [1.36952104]
error [-1.64417417]
error [-1.62773243]
error [-3.61145511]
error [-2.57534056]
error [-3.54958715]
error [4.48590872]
error [0.44104963]
error [0.43663914]
error [-3.56772725]
error [4.46795002]
error [1.42327052]
error [4.40903781]
error [4.36494744]
error [3.32129796]
error [2.28808498]
error [2.26520413]
error [-2.75744791]
error [1.27012657]
error [3.2574253]
error [0.22485105]
error [1.22260254]
error [-1.78962348]
error [2.22827275]
error [-2.79400998]
error [-0.76606988]
error [-3.75840918]
error [2.27917491]
error [4.25638316]
error [3.21381933]
error [3.18168114]
error [4.14986433]
error [2.1083656

ValueError: ignored